# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [1]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}

2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [2]:
'''obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])
obj'''

'obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])\nobj'

3. Разбейте текст формулировки задачи 2 на слова.

## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [3]:
import pandas as pd

In [4]:
recipes = pd.read_csv('recipes_sample.csv')
new = recipes[['id', 'minutes']].sample(n=5)
new

,id,minutes
22108,466289,675
5205,481304,32
1767,504083,23
25390,327226,35
14724,464317,60


In [5]:
!pip install tabulate

In [6]:
from tabulate import tabulate
print(tabulate(new, headers='keys', showindex = False, tablefmt='github'))

|     id |   minutes |
|--------|-----------|
| 466289 |       675 |
| 481304 |        32 |
| 504083 |        23 |
| 327226 |        35 |
| 464317 |        60 |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [7]:
from bs4 import BeautifulSoup as bs

In [8]:
with open('steps_sample.xml', 'r') as f: 
    steps_sample = bs(f, 'xml')

In [9]:
def show_info(n):
    res = f'{recipes[recipes.id == n].name.iloc[0].title()} \n \n'
    for r in steps_sample.recipes.find_all('recipe'):
        if r.find('id').next == str(n):
            steps = [step.next for step in r.steps.find_all('step')]
    for i, step in enumerate(steps):
        res += f'{i + 1}. {step.capitalize()} \n'
        
    res += '\n' + '--'*10 + '\n' + '\n' + f'Автор:  {recipes[recipes.id == n].contributor_id.iloc[0]}' + '\n' + f'Среднее время приготовления:  {recipes[recipes.id == n].minutes.iloc[0]} минут'
    return print(f'{res}')

In [10]:
show_info(170895)

Leeks And Parsnips  Sauteed Or Creamed 
 
1. Clean the leeks and discard the dark green portions 
2. Cut the leeks lengthwise then into one-inch pieces 
3. Melt the butter in a medium skillet , med 
4. Heat 
5. Add the garlic and fry 'til fragrant 
6. Add leeks and fry until the leeks are tender , about 6-minutes 
7. Meanwhile , peel and chunk the parsnips into one-inch pieces 
8. Place in a steaming basket and steam 'til they are as tender as you prefer 
9. I like them fork-tender 
10. Drain parsnips and add to the skillet with the leeks 
11. Add salt and pepper 
12. Gently sautee together for 5-minutes 
13. At this point you can serve it , or continue on and cream it: 
14. In a jar with a screw top , add the half-n-half and arrowroot 
15. Shake 'til blended 
16. Turn heat to low under the leeks and parsnips 
17. Pour in the arrowroot mixture , stirring gently as you pour 
18. If too thick , gradually add the water 
19. Let simmer for a couple of minutes 
20. Taste to adjust seasoning

In [11]:
recipes[recipes.id == 170895]

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
15754,leeks and parsnips sauteed or creamed,170895,27,8377,2006-05-31,21.0,this is good sauteed only or creamed. very eas...,9.0


## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [12]:
import re
for r in steps_sample.recipes.find_all('recipe'):
    if r.find('id').next == '25082':
        steps = [step.next for step in r.steps.find_all('step')]
for i, step in enumerate(steps):
    print(re.findall(r'\d+\shours*', step))
    print(re.findall(r'\d+\sminutes*', step))

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['20 minutes']
[]
[]
[]
['10 minutes']
[]
[]
['2 hours']
[]
[]
[]
[]
[]
[]
[]
[]
['10 minutes']
[]
[]
[]
[]
[]
['20 minutes', '30 minutes']
[]
[]


4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [13]:
res = []
for i in range(30000):
    if type(recipes.description.iloc[i]) is str:
        res += re.findall(r'\bthis\D+,\s*but', recipes.description.iloc[i])
ress = [i for i in res if len(i)>1]
print(len(ress))
print(ress[8:11])

2041
['this in a magazine, yet to try, but', 'this up, but', 'this is a great meal eaten the same day ,but even better the next day , if you can wait! add your favourite spices, but']


5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [14]:
dict_rec = []
step = []
for reciep in steps_sample.recipes.find_all('recipe'):
    steps = [step.next for step in reciep.steps.find_all('step')]
    dict_rec.append({reciep.find('id').next: steps})
    if reciep.find('id').next == '72367':
        step = steps
result = []
for i in step:
    step_ = re.findall(r'\b\d+\s*/\s*\d+\b' , i)
    for j in step_:
        step_new = j.replace(' ', '')
    if len(step_)==1:
        result.append(re.sub(step_[0], step_new, i))
    else:
        result.append(i)
result

['mix butter , flour , 1/3 c',
 'sugar and 1-1/4 t',
 'vanilla',
 'press into greased 9" springform pan',
 'mix cream cheese , 1/4 c',
 'sugar , eggs and 1/2 t',
 'vanilla beating until fluffy',
 'pour over dough',
 'combine apples , 1/3 c',
 'sugar and cinnamon',
 'arrange on top of cream cheese mixture and sprinkle with almonds',
 'bake at 350 for 45-55 minutes , or until tester comes out clean']

### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [15]:
import nltk
from nltk.tokenize import WordPunctTokenizer

In [16]:
tkz = WordPunctTokenizer()
voc = set()
steps_vraw = steps_sample.find_all('steps')
raw_steps = [list(filter(lambda x: len(x) > 0, i.get_text().split('\n'))) for i in steps_vraw]
raw_steps = [list(filter(lambda z: z.isalpha(), i)) for i in raw_steps]
for i in raw_steps:
    voc.update(set(tkz.tokenize(' '.join(i).lower())))
len(voc)

378

7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [17]:
from nltk.tokenize import sent_tokenize

In [18]:
lengths = recipes.description.apply(lambda x: len(sent_tokenize(str(x))))

In [19]:
recipes.iloc[lengths.argsort().values[-6:-1][::-1]]

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
481,alligator claws avocado fritters with chipot...,287008,45,765354,2008-02-19,NaN,a translucent golden-brown crust allows the gr...,9.0
22566,rich barley mushroom soup,328708,60,221776,2008-10-03,NaN,this is one of the best soups i've ever made a...,10.0
6779,chocolate tea,205348,6,428824,2007-01-14,NaN,i wrote this because there are an astounding l...,NaN
16296,little bunny foo foo cake carrot cake with c...,316000,68,689540,2008-07-27,14.0,the first time i made this cake i grated a mil...,NaN
14136,hot and sour soup betty foo hunan restaurant,141637,35,167236,2005-10-17,NaN,this is the recipe as taught in the main line ...,19.0


8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [20]:
from nltk.tag import pos_tag

In [21]:
def tags(s):
    tags_ = pos_tag(tkz.tokenize(s))
    paddings = [max(len(i), len(k)) for i, k in tags_]

    upper = ''
    lower = ''
    for p, (i, t) in zip(paddings, tags_):
        upper += f"{t:^{p+2}}"
        lower += f"{i:^{p+2}}"
    print(upper)
    print(lower)
tags(recipes[recipes.id == 241106].name.values[0])

    JJ      NNS     IN      NNS     VBP     JJ    CC    JJ     NNS   
 eggplant  steaks  with  chickpeas  feta  cheese  and  black  olives 
